### Závěrečný projet - modul Python

V tomto projektu budete analyzovat data ze stránky `www.sreality.cz`. Budete tedy získávat aktuální nabídku bytů ve Zlíně.
Projekt se dělí na 2 části:
1. WebScraping - pomocí knihovny `BeautifulSoup` získejte všechny aktuální inzerce bytů v daném městě. 
- Získáte **tyto** informace: `URL odkaz bytu, rozměry bytu, kompozice bytu, cena bytu, lokace bytu (Ulice+město)`
2. Datová analýza - ETL proces
- V první fázi prověďte processing dat tak, aby obsahovala příslušné datové typy, nenulové hodnoty atp. Ulice i město bude v samostatných sloupcích.
- Zprocesovaná a vyčištěná data vyexportujte do `.csv` souboru s názvem `{vašeJméno}_zdrojová_data.csv`
- Druhou část začněte tak, že si do dataframe nahrajete vámi vyexportovaný `.csv` soubor s vyscrapovanými daty.
3. Datová analýza - analýza dat
- Ve druhé fázi proveďte analýzu dat a odpovězte na tyto dotazy:
- `Jaká je průměrná cena bytů?`
- `Jaká je průměrná cena bytů pro každou kompozici (1+1, 2+1, atp.)? Zobrazte i na grafu.`
- `Jaká je průměrná velikost bytu pro každou kompozici?`
- `Existuje ulice, kde je vyšší koncentrace dražších bytů?`
- `Jaký typ kompozice je v daném městě nejčastěji inzerovan? Který to je? Proč myslíte, že tomu tak je?`
- `Existují zde inzerce bytů, které stojí více, než 20.000,-? Pokud ano, jsou v této cenové hladině inzerovány i maximálně dvoupokojové byty (2+1/2+kk)?`
- `Zjištěte minimální a maximální inzerce pro každou kompozici bytu. Která kompozice má největší rozptyl mezi minimální a maximální inzerovanou cenou?`


Dotazy zodpovězte ve vašem pracovním notebooku a výsledný soubor nahrajte na GitHub.


### WebScraping

In [57]:

from bs4 import BeautifulSoup
import requests
import pandas as pd

strana = 1

while True:
        mesto = 'zlin'
        data = requests.get(f'https://www.sreality.cz/hledani/pronajem/byty/{mesto}?strana={strana}').text
        soup = BeautifulSoup(data, 'html.parser')

        byty=[]
        list_inzeratu = []

        list_inzeratu = soup.findAll('li', 'MuiGrid-root MuiGrid-item css-l1328q')

        for objekt_byt in list_inzeratu:
    
            # Kontrola reklamy
            jeInzerat = objekt_byt.find('div', 'MuiBox-root css-n6y9a7')
            if jeInzerat is None:
                continue
    
            cena = objekt_byt.find('p', class_='MuiTypography-root MuiTypography-body1 css-1ndcg2e').text
            rozmer_bytu = objekt_byt.find('p', class_='MuiTypography-root MuiTypography-body1 css-13ztabn').text
            adresa = objekt_byt.findAll('p', class_='MuiTypography-root MuiTypography-body1 css-13ztabn')[1].text


   
            objekt_byt = {
                'Cena': cena,
                'Rozměr bytu': rozmer_bytu,
                'Adresa': adresa,
                }
            byty.append(objekt_byt)
        """
        ------------------------------------------------------------------------------------------------
        Tohle ponechte pod vaším kódem - kontroluje se, zda na stránce existuje tlačítko Další stránky. Pokud ne, program končí.
        ------------------------------------------------------------------------------------------------
        """               
        button = soup.find('button', 'MuiButtonBase-root MuiButton-root MuiButton-outlined MuiButton-outlinedInherit MuiButton-sizeMedium MuiButton-outlinedSizeMedium MuiButton-colorInherit MuiButton-root MuiButton-outlined MuiButton-outlinedInherit MuiButton-sizeMedium MuiButton-outlinedSizeMedium MuiButton-colorInherit css-lp5ywq').text

        if button != 'Další stránka':
            print('Již zde není žádná další strana. Končím')
            break

        strana += 1
        
        print(f'Scrapuju stránku č. {strana}')    
        
df = pd.DataFrame(byty)
df

Scrapuju stránku č. 2
Scrapuju stránku č. 3
Scrapuju stránku č. 4
Scrapuju stránku č. 5
Scrapuju stránku č. 6
Scrapuju stránku č. 7
Scrapuju stránku č. 8
Již zde není žádná další strana. Končím


,Cena,Rozměr bytu,Adresa
0,22 000 Kč/měsíc,Pronájem bytu 2+kk 85 m²,Luhačovice
1,12 000 Kč/měsíc,Pronájem bytu 1+1 40 m²,Racková
2,20 000 Kč/měsíc,Pronájem bytu 2+kk 58 m²,"Bartošova, Zlín"
3,18 000 Kč/měsíc,Pronájem bytu 2+kk 49 m²,"Tyršovo nábřeží, Zlín"
4,19 000 Kč/měsíc,Pronájem bytu 4+1 79 m²,"Na Honech II, Zlín"
5,39 900 Kč/měsíc,Pronájem bytu 4+kk 100 m²,Luhačovice
6,9 000 Kč/měsíc,Pronájem bytu 2+kk 40 m²,"V Dolině, Zlín - Jaroslavice"
7,14 900 Kč/měsíc,Pronájem bytu 2+kk 47 m²,"K Luhám, Zlín - Louky"


In [54]:
df.to_csv('webscrap_data_sreality.csv')

In [55]:
import pandas as pd
df = pd.read_csv('webscrap_data_sreality.csv')
df

,Unnamed: 0,Cena,Rozměr bytu,Adresa
0,0,19 000 Kč/měsíc,Pronájem bytu 3+1 73 m²,"Slezská, Zlín"
1,1,12 000 Kč/měsíc,Pronájem bytu 1+1 40 m²,Racková
2,2,20 000 Kč/měsíc,Pronájem bytu 2+kk 58 m²,"Bartošova, Zlín"
3,3,18 000 Kč/měsíc,Pronájem bytu 2+kk 49 m²,"Tyršovo nábřeží, Zlín"
4,4,19 000 Kč/měsíc,Pronájem bytu 4+1 79 m²,"Na Honech II, Zlín"
5,5,39 900 Kč/měsíc,Pronájem bytu 4+kk 100 m²,Luhačovice
6,6,9 000 Kč/měsíc,Pronájem bytu 2+kk 40 m²,"V Dolině, Zlín - Jaroslavice"
7,7,14 900 Kč/měsíc,Pronájem bytu 2+kk 47 m²,"K Luhám, Zlín - Louky"


### Datová analýza

In [56]:
df['Cena'] = df['Cena'].str.replace('Kč/měsíc', '').str.replace('\u00A0', '').astype(int)
df['Rozměr bytu'] = df['Rozměr bytu'].str.replace('Pronájem bytu ','')
df['Kompozice'] = df['Rozměr bytu'].str.split(' ', n=2, expand=True)[0]
df['Rozměry v m2'] = df['Rozměr bytu'].str.split(' ', n=2, expand=True)[1].str.replace('m²','').astype(int)
df.head()

,Unnamed: 0,Cena,Rozměr bytu,Adresa,Kompozice,Rozměry v m2
0,0,19000,3+1 73 m²,"Slezská, Zlín",3+1,73
1,1,12000,1+1 40 m²,Racková,1+1,40
2,2,20000,2+kk 58 m²,"Bartošova, Zlín",2+kk,58
3,3,18000,2+kk 49 m²,"Tyršovo nábřeží, Zlín",2+kk,49
4,4,19000,4+1 79 m²,"Na Honech II, Zlín",4+1,79
